In [7]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
df = pd.read_csv('homework_4.1.csv')

df.head(), df.columns.tolist()

(   Unnamed: 0  Z         W         X         Y
 0           0  0 -0.155644 -0.496971  0.282484
 1           1  1  0.529539  2.284240  4.740596
 2           2  1  0.910514  0.872232  3.449569
 3           3  1 -0.705476  2.157260  3.002531
 4           4  0 -0.590874 -0.386730 -1.848796,
 ['Unnamed: 0', 'Z', 'W', 'X', 'Y'])

In [3]:
# First, calculate the average Y and X for Z = 1 and Z = 0
mean_Y_Z1 = df[df['Z'] == 1]['Y'].mean()
mean_Y_Z0 = df[df['Z'] == 0]['Y'].mean()
mean_X_Z1 = df[df['Z'] == 1]['X'].mean()
mean_X_Z0 = df[df['Z'] == 0]['X'].mean()

# Compute the Wald estimator (effect = delta Y / delta X)
delta_Y = mean_Y_Z1 - mean_Y_Z0
delta_X = mean_X_Z1 - mean_X_Z0
wald_estimate = delta_Y / delta_X

wald_estimate

np.float64(1.5618587073955674)

In [4]:
# Define bins over W (e.g., 10 equal-width bins)
num_bins = 10
df['W_bin'] = pd.cut(df['W'], bins=num_bins)

# For each bin, compute the Wald estimator (delta Y / delta X)
effects = []

for _, group in df.groupby('W_bin'):
    mean_Y_Z1 = group[group['Z'] == 1]['Y'].mean()
    mean_Y_Z0 = group[group['Z'] == 0]['Y'].mean()
    mean_X_Z1 = group[group['Z'] == 1]['X'].mean()
    mean_X_Z0 = group[group['Z'] == 0]['X'].mean()

    delta_Y = mean_Y_Z1 - mean_Y_Z0
    delta_X = mean_X_Z1 - mean_X_Z0

    # Avoid division by zero or missing values
    if np.isfinite(delta_Y) and np.isfinite(delta_X) and delta_X != 0:
        effects.append(delta_Y / delta_X)

# Average the local Wald estimates
average_local_effect = np.mean(effects)
average_local_effect

/var/folders/2r/46mxdql52bn78bp6pr2cps340000gn/T/ipykernel_40111/2614700569.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for _, group in df.groupby('W_bin'):


np.float64(0.9085420555919094)

In [6]:
# Load both datasets
df_a = pd.read_csv("homework_4.2.a.csv")
df_b = pd.read_csv("homework_4.2.b.csv")

# Show the first few rows and column names of each to verify structure
df_a.head(), df_a.columns.tolist(), df_b.head(), df_b.columns.tolist()

(   Unnamed: 0          X  Y
 0           0  81.822339  1
 1           1  92.487870  0
 2           2  85.372460  0
 3           3  78.828025  0
 4           4  75.807080  1,
 ['Unnamed: 0', 'X', 'Y'],
    Unnamed: 0         X2  Y2
 0           0  76.643034   1
 1           1  87.743397   1
 2           2  81.639469   1
 3           3  73.740485   0
 4           4  90.480268   1,
 ['Unnamed: 0', 'X2', 'Y2'])

In [8]:
# Define a function to perform linear regression on either side of the cutoff
def analyze_rdd(df, score_col, outcome_col, cutoff=80):
    below = df[df[score_col] < cutoff]
    above = df[df[score_col] >= cutoff]

    X_below = sm.add_constant(below[score_col])
    y_below = below[outcome_col]
    model_below = sm.OLS(y_below, X_below).fit()

    X_above = sm.add_constant(above[score_col])
    y_above = above[outcome_col]
    model_above = sm.OLS(y_above, X_above).fit()

    return {
        "slope_below": model_below.params[score_col],
        "slope_above": model_above.params[score_col],
        "r2_below": model_below.rsquared,
        "r2_above": model_above.rsquared
    }

# Apply to both datasets
results_a = analyze_rdd(df_a, "X", "Y")
results_b = analyze_rdd(df_b, "X2", "Y2")

results_a, results_b

({'slope_below': np.float64(0.00022383079703675596),
  'slope_above': np.float64(0.00016108686482244804),
  'r2_below': np.float64(8.623788795625309e-06),
  'r2_above': np.float64(3.357626736422148e-06)},
 {'slope_below': np.float64(0.010216711274456688),
  'slope_above': np.float64(0.005008614068236688),
  'r2_below': np.float64(0.016161139360567178),
  'r2_above': np.float64(0.013428593795579924)})